In [ ]:
import collections
import math
import functools
import io
import random
import os
import requests
import zipfile
from random import sample
from typing import List, Optional, Tuple
import matplotlib as plt
import numpy as np
import pandas as pd
import time
from timeit import default_timer as timer
import numba 
from sklearn.metrics import ndcg_score
from numba import njit
import csv

In [ ]:
ratings_df = pd.read_csv('train_movieLens.csv')
ratings_df.columns = ['UserID', 'MovieID', 'Rating']

UserList = ratings_df['UserID'].values
ItemList = ratings_df['MovieID'].values
unique_users = np.unique(UserList)
#print(ratings_df)
print('md..',max(np.unique(ItemList)))
Test_ratings_df = pd.read_csv('valid_movieLens.csv')
Test_ratings_df.columns = ['UserID', 'MovieID', 'Rating']
print(Test_ratings_df.shape)
UserList_Test=Test_ratings_df['UserID'].values
ItemList_Test=Test_ratings_df['MovieID'].values
unique_users_Test=np.unique(UserList_Test)
print('..',max(unique_users_Test))


Full_ratings_df = pd.read_csv('movieLens_full.csv')
Full_ratings_df.columns = ['UserID', 'MovieID', 'Rating']
print(len(ratings_df))
print(len(Test_ratings_df))
print(len(Full_ratings_df))

In [ ]:
all_r , p = ratings_df.shape
print(all_r)
all_r_Test , p = Test_ratings_df.shape
K=20
N = len(set(Full_ratings_df.UserID))+1
M = len(set(Full_ratings_df.MovieID))+1
print(N)
print(M)
print("No. of users : ",len(set(Full_ratings_df.UserID)))
print("No. of Movies: ",len(set(Full_ratings_df.MovieID)))

N_test1= max(set(Test_ratings_df.UserID))
M_test1=max(set(Test_ratings_df.MovieID))
print(N_test1)
print(M_test1)
Test_ratings_df.shape

In [ ]:
popular_df = pd.read_csv('movieLens_popular.csv')
pop = popular_df.values.tolist()


In [ ]:
nonpopular_df = pd.read_csv('movieLens_non_popular.csv')
nonpop = nonpopular_df.values.tolist()

In [ ]:
item_list = {}

for movie_no in range(M):
    item_list[movie_no] = 0
    
for index, row in ratings_df.iterrows():
    idMovie = row['MovieID']
    item_list[idMovie]+=1


In [ ]:

Test_item_list = {}
for movie_no in range(M_test1+2):
    Test_item_list[movie_no] = 0
        
        
for index, row in Test_ratings_df.iterrows():
    idMovie = row['MovieID']
    
    Test_item_list[idMovie]+=1        


In [ ]:
ratings_df = ratings_df.pivot( 'UserID' , 'MovieID' , 'Rating').fillna(0)
ratings_df = np.array(ratings_df)

Test_ratings_df = Test_ratings_df.pivot( 'UserID' , 'MovieID' , 'Rating').fillna(0)
Test_ratings_df = np.array(Test_ratings_df)

print(Test_ratings_df.shape)

N_test,M_test= Test_ratings_df.shape

In [ ]:
dictt = []
for i in range(M):#3666
    if [i] in pop:
        dictt.append(1)
    else:
        dictt.append(0)
dictt = np.array(dictt)


In [ ]:
@njit(parallel=False)
def m_f(R, P, Q, K,dictt, steps=100, alpha=0.000128, beta = 0.04): #function for matrix factorization
  m_error = []
  pm_error = []
  npm_error = []
  biasm = []
  #ARP_ = []

  Q = Q.T #taking transpose
 # print('Step A')
  for step in range(steps):
   # if(step%8 == 0):
    #  print("====",step)
    for i in range(N): #for every row
      for j in range(M): #for every column
        if R[i][j] > 0: #if user has rated the item
          eij = R[i][j] - np.dot(P[i,:] , Q[:,j]) #error = actual rating - predicted rating
          #calculate gradient with alpha and beta parameter
          for k in range(K): #for kth feature
            P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j]  - beta*P[i][k] )
            Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k]  - beta*Q[k][j] )
    #print(" P :",P)
    #print(" Q :",Q)
    eR = np.dot(P,Q)
    e = 0
    count = 0
    np_count = 0
    np_error = 0
    p_error = 0
    p_count = 0
    #print('Step B')
    for i in range(N):#prediction
      for j in range(M):
        if R[i][j]> 0:
            #print(R[i][j])
            err = np.round(pow(R[i][j] - np.dot(P[i,:],Q[:, j]), 2) , 3)

            if dictt[j] == 0:
                np_count +=1
                np_error += err
            #if the item is popular
            else:
                p_count +=1
                p_error += err
            
            count += 1
            e = e + err
          
            


    #print("main error : ",np.round(np.sqrt(e / count) ,3))
    m_error.append(np.round((e / count) ,3))

    #print("loss on popular items : ",np.round_(np.sqrt(p_error/p_count) , 3))
    pm_error.append(np.round((p_error/p_count) , 3))

    #print("loss on non popular items : ",np.round_(np.sqrt(np_error/np_count) , 3))
    npm_error.append(np.round((np_error/np_count) , 3))

    #print("popularity bias: ", np.round((  ((1/np_count)*np_error) - ((1/p_count)*p_error)),3) )
    biasm.append(np.round((  ((1/np_count)*np_error) - ((1/p_count)*p_error)),3))


  
            
        
  return P,Q.T,m_error,pm_error,npm_error,biasm,eR




In [ ]:

@njit(parallel=False)
def calc_ndcg_score(y_true, y_score):

    y_max_sorted = y_true[y_true.argsort()[::-1]]
    y_true_sorted = y_true[y_score.argsort()[::-1]]

    k = y_true.shape[0]
   

    dcg_score = y_true_sorted[0] - 1
    for i in np.arange(1, k):
        dcg_score += y_true_sorted[i] / np.log2(i + 1)

    max_score =2**(y_max_sorted[0])- 1
    for i in np.arange(1, k):
        max_score += y_max_sorted[i] / np.log2(i + 1)
    
    if max_score!=0:
        ndcg = dcg_score / max_score
    else:
        ndcg = 0 
    return ndcg 
    

In [ ]:
@njit(parallel=False)
def calcARP(UserList,ItemList,unique_users,eR,R):
    
    arp_new = 0
    
    w_count = 0 
    ndcg_users = []
    cnt = 0 
    for u in unique_users:
        pred_user = []
        arp_user= 0
        for i,j in zip(UserList,ItemList):
            if i ==u and i<N and j<M:
                predRating = eR[i][j]
                pred_user.append(tuple((j,predRating)))
        pred_user=sorted(pred_user, key = lambda t: t[1])
        pred_user.reverse()
        pred_user=pred_user[:10]
        
        true_y=[]
        pred_y=[]
        
        w_user = 0 
        for each in pred_user:
            w_i = len(np.where(ItemList==each[0])[0])
            true_y.append(R[u][each[0]])
            pred_y.append(each[1])
            w_user+= w_i
            
        ndcg_score_user = calc_ndcg_score(np.array(true_y),np.array(pred_y))#ndcg_score([true_y],[pred_y])
        ndcg_users.append(ndcg_score_user)
        cnt +=1
        w_count += w_user
    deno = 10*len(UserList)*len(unique_users)
   # print('Deno is ',deno)
    arp_new = np.round(w_count /deno,7)
    
    sum_=0
    for each in ndcg_users:
        sum_+= each
   # print('SUm is ',sum_,cnt)    
    ndcg_final = sum_/cnt
    #ndcg_final = #np.mean(ndcg_users)
    
    return arp_new,ndcg_final


    
@njit(parallel=False)
def calcARP_Test(UserList_Test,ItemList_Test,unique_users_Test,eR_Test,Test_R):
    
    arp_new = 0
    
    w_count = 0 
    ndcg_users = []
    cnt = 0 
    for u in unique_users_Test:
        pred_user = []
        arp_user= 0
        for i,j in zip(UserList_Test,ItemList_Test):
            if i ==u and i<N_test and j<M_test:
                predRating = eR_Test[i][j]
                pred_user.append(tuple((j,predRating)))
        pred_user=sorted(pred_user, key = lambda t: t[1])
        pred_user.reverse()
        pred_user=pred_user[:10]
        
        true_y=[]
        pred_y=[]
        
        w_user = 0 
        for each in pred_user:
            w_i = len(np.where(ItemList==each[0])[0])
            true_y.append(Test_R[u][each[0]])
            pred_y.append(each[1])
            w_user+= w_i
            
        ndcg_score_user = calc_ndcg_score(np.array(true_y),np.array(pred_y))#ndcg_score([true_y],[pred_y])
        ndcg_users.append(ndcg_score_user)
        cnt +=1
        w_count += w_user
    deno = 10*len(UserList_Test)*len(unique_users_Test)
   # print('Deno is ',deno)
    arp_new = np.round(w_count /deno,7)
    
    sum_=0
    for each in ndcg_users:
        sum_+= each
   # print('SUm is ',sum_,cnt)    
    ndcg_final = sum_/cnt
    #ndcg_final = #np.mean(ndcg_users)
    
    return arp_new,ndcg_final
    
    

In [ ]:
N,M = ratings_df.shape
K = 20
print("N_users : ",N)
print("N_items : ",M)
print("N_Dimensions : ", K)
#pred = {}
R = np.array(ratings_df)
Test_R = Test_ratings_df
seeds = [0,100,200,300,400,500,600,700,800,900]


list_pop_error=[]
list_non_pop_error=[]
list_ndcg=[]
list_arp=[]
list_error=[]
list_bias=[]

list_pop_error_test=[]
list_non_pop_error_test=[]
list_ndcg_test=[]
list_arp_test=[]
list_error_test=[]
list_bias_test=[]
for run in range(10):
    np.random.seed(seeds[run])
    random.seed(seeds[run])
    Q = np.round( np.random.uniform(low =0.1 , high = 0.9, size = (M,K)) ,2) 
    P = np.round(np.random.uniform(low =0.1 , high = 0.9, size = (N,K)),2)

    nP, nQ , m_error,pm_error,npm_error,biasm , eR= m_f(R,P,Q,K,dictt)

    trainoverepoch = pd.DataFrame(list(zip(m_error , pm_error , npm_error , biasm )),
               columns =['Overall_loss', 'Error on popular items' , 'Error on non-popular items' , 'Popularity bias' ])
    #pred[run] = eR
    trainoverepoch.to_csv('output_new_mf_reg_movielens_train_overepoch'+str(run)+'_.csv')
    print(run)
 
    ans = calcARP(UserList,ItemList,unique_users,eR,R)
    ARP,NDCG = ans[0], ans[1]
    trainoverrun = []
    trainoverrun.append(m_error[-1])
    trainoverrun.append(pm_error[-1])
    trainoverrun.append(npm_error[-1])
    trainoverrun.append( biasm[-1])
    trainoverrun.append(np.round(ARP,6))
    trainoverrun.append(np.round(NDCG,6))
    print(trainoverrun)
    list_pop_error.append(pm_error[-1])
    list_non_pop_error.append(npm_error[-1])
    list_ndcg.append(np.round(NDCG,6))
    list_arp.append(np.round(ARP,6))
    list_error.append(m_error[-1])
    list_bias.append(biasm[-1])
    
    
    
    testoverrun = []
    
    
    nP = nP
    nQ = nQ.T
    #############################  Testing  ################################
    Test_e = 0
    Test_count = 0
    Test_np_count = 0
    Test_np_error = 0
    Test_p_error = 0
    Test_p_count = 0
    
    for i in range(N_test):#prediction
      for j in range(M_test):
        if Test_R[i][j]> 0:
            #print(R[i][j])
            #print('R i j ',Test_R[i][j])

            Test_err = np.round(pow(Test_R[i][j] - np.dot(nP[i,:],nQ[:, j]), 2) , 3)
            
            
            if dictt[j] == 0:
                Test_np_count +=1
                Test_np_error += Test_err
            #if the item is popular
            else:
                Test_p_count +=1
                Test_p_error += Test_err
            
            Test_count += 1
            Test_e = Test_e + Test_err
            
    eR_Test= np.array(np.dot(nP,nQ))
    ans = calcARP_Test(UserList_Test,ItemList_Test,unique_users_Test,eR_Test,Test_R)    #calcARP_Test(eR_Test,Test_R)
    Test_ARP,NDCG_Test = ans[0],ans[1]
        
            
    testoverrun.append(np.round((Test_e / Test_count) ,6))
    testoverrun.append(np.round((Test_p_error/Test_p_count) , 6))
    testoverrun.append(np.round((Test_np_error/Test_np_count) , 6))
    testoverrun.append(np.round((  ((1/Test_np_count)*Test_np_error) - ((1/Test_p_count)*Test_p_error)),6))
    testoverrun.append(np.round(Test_ARP,6))
    testoverrun.append(np.round(NDCG_Test,6))
    print('Test : ',testoverrun)
    list_pop_error_test.append(np.round((Test_p_error/Test_p_count) , 6))
    list_non_pop_error_test.append(np.round((Test_np_error/Test_np_count) , 6))
    list_ndcg_test.append(np.round(NDCG_Test,6))
    list_arp_test.append(np.round(Test_ARP,6))
    list_error_test.append(np.round((Test_e / Test_count) ,6))
    list_bias_test.append(np.round((  ((1/Test_np_count)*Test_np_error) - ((1/Test_p_count)*Test_p_error)),6))
    
    
    
    with open('output_new_mf_reg_movieLens_train_overrun', 'w') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(trainoverrun)
      wr.writerow(testoverrun)

print(np.mean(list_pop_error))
print(np.mean(list_non_pop_error))
print(np.mean(list_ndcg))
print(np.mean(list_arp))
print(np.mean(list_error))
print(np.mean(list_bias))

print()
print(np.mean(list_pop_error_test))
print(np.mean(list_non_pop_error_test))
print(np.mean(list_ndcg_test))
print(np.mean(list_arp_test))
print(np.mean(list_error_test))
print(np.mean(list_bias_test))

print('-'*10)

print(np.std(list_pop_error))
print(np.std(list_non_pop_error))
print(np.std(list_ndcg))
print(np.std(list_arp))
print(np.std(list_error))
print(np.std(list_bias))
print()
print(np.std(list_pop_error_test))
print(np.std(list_non_pop_error_test))
print(np.std(list_ndcg_test))
print(np.std(list_arp_test))
print(np.std(list_error_test))
print(np.std(list_bias_test))

    
    
    
#     with open('output_mf_train_overrun', 'w') as myfile:
#       wr = csv.writer(myfile)
#       wr.writerow(trainoverrun)
          

    #train
    #alpha=0.0023, beta = 0.02